In [1]:
import os
import pandas as pd

In [2]:
os.chdir("/home/rocassius/w266_final/scripts/assembly")
from constant import DOC_SAMPLE_PATH, DOC_PROPER_PATH
from document import *

In [3]:
os.chdir("/home/rocassius/w266_final/scripts/modeling")
from rmn import *
from analysis import *
from rmn_analyzer import *
from rmn_data_generator import *

In [4]:
df_all = load_documents(sessions=list(range(104,105)), read_path=DOC_PROPER_PATH)

In [5]:
rmn = RMN()
rmn.load_rmn(name="full", save_path = "/home/rocassius/gen-data/models")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### Analyze the 105th session with an RMN_Analyzer

In [6]:
analyzer = RMN_Analyzer(rmn, df_all)

In [7]:
# compute inter-party divergence on abortion
analyzer.inter_party_divergence(subject = 'abortion', n=10000)

23/23 [==============================] - 7s 287ms/step


{'estimate': 0.11111702782010845,
 'lower': 0.1092181655534295,
 'upper': 0.11301589008678742}

In [8]:
analyzer.intra_party_divergence(party = 'D', subject = 'abortion', n=10000)

{'estimate': 0.11041956343874594,
 'lower': 0.10988138881172664,
 'upper': 0.11095773806576524}

In [9]:
analyzer.intra_party_divergence(party = 'R', subject = 'abortion', n=10000)

{'estimate': 0.11056319454955117,
 'lower': 0.1100330910804315,
 'upper': 0.11109329801867085}

In [11]:
analyzer.intra_party_hh(party = 'R', subject = 'abortion', n=10000)

{'estimate': 0.011164854280650616,
 'lower': 0.011153814170616795,
 'upper': 0.01117589358136512}